In [6]:
# the packages
import pandas as pd
import numpy as np
import networkx as nx

# import my own helper functions
from read import read_sims_result
from clean import cleanup_0IR_exp
from clean import cleanup_network

# pearson correlation coeffcient
from scipy.stats.stats import pearsonr

# deque
from collections import deque

# deep copy
from copy import deepcopy

# random [0,1)
from random import random

# Page Rank
from networkx.algorithms.link_analysis.pagerank_alg import pagerank, pagerank_numpy, pagerank_scipy

# distance
from networkx.algorithms.shortest_paths.generic import shortest_path_length

# logistic regression
from sklearn.linear_model import LogisticRegression

# cross validation
from sklearn.model_selection import KFold

# confusion matrix
from sklearn.metrics import confusion_matrix

# plot
import matplotlib.pyplot as plt

In [7]:
# independent variables
independent = ["deposits", "cash", "assets", "credit available", "wealth", "leverage", 
         "dummy-0-leverage",
         "wealth-lag", "deposits-lag", "cash-lag", "assets-lag", "leverage-lag", 
         "credit-available-lag", "credit-issued-lag", "dummy-0-leverage-lag",
         "over-leverage-frequency"]

In [8]:
# ###########################
# Read OIR results, and fit the model
# ###########################
df0 = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0622/0IR300s", 32)
df0c = cleanup_0IR_exp(df0, numNode=32, numPeriod=15, numSim=100, balanced=True)

X = df0c[independent]
y = df0c["default-next"]

final = LogisticRegression(penalty="l1", C=0.007)
final.fit(X,y)

LogisticRegression(C=0.007, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [9]:
# make sure no defaults in 0IR
# sum(df0[df0["defaults due to interest"]
#     +df0["defaults due to negative wealth"]
#     +df0["defaults due to deposit shock"] == 0].loc[:,"dot0":"dot30"].values)

In [10]:
# ###########################
# Read & process positive IR results
# ###########################
df_1 = read_sims_result("/Users/xcheng/Documents/Oberlin/Summer2/DataAnalysis/data/0625/1IR", 32)
mx_1n = cleanup_network(df_1, numNode=32, numPeriod=15, numSim=50)
df_1c = cleanup_0IR_exp(df_1, numNode=32, numPeriod=15, numSim=50)

In [11]:
def create_edge_weight(N, dff, mid):
    """
    Calculate weight for edges
    Each debt is divided by lenders' wealth (w/o haircut)，
    The result number r is scaled to [0, 1) using g(r)=r/(mid+r)
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    df: Pandas dataframe (no cleanup)
        where we get banks' wealth (w/o haircut)
    mid: int
        the debt-to-wealth ratio resulting in 50% probability of spreading default
    
    Returns
    ----------
    WN: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        new weighted debt adjacency matrices
    """
    
    WN = np.copy(N)
    simNum, periodNum, bankNum, _= N.shape
    dff["book_wealth"] = (dff["assets"] + dff["cash"] + dff["debt owed"] 
                         - dff["debt to pay"] - dff["deposits"])
    
    for s in range(simNum):
        for p in range(1,periodNum+1):
            for lender in range(bankNum):
                w = dff[np.array(dff["sim#"]==s) &
                        np.array(dff["period"]==p) & 
                        np.array(dff["bankID"]==lender)
                       ]["book_wealth"].values[0]
                
                # helper function
                def f(a):
                    if a > 0: # there is debt
                        if w > 0: # positive wealth
                            t = a/w
                            return t/(t+mid)
                        else: # 0 or negative wealth
                            return 100/(100+mid)
                    else: # no debt or weird data 
                        return 0
                
                WN[s, p-1, :, lender] = [f(k) for k in WN[s, p-1, :, lender]]
                    
    return WN

In [12]:
def create_node_weight(N, dff, model, variables):
    """
    Calculate weight for nodes
    Each debt is multiplied by lenders' predicted default probability
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    dff: Pandas dataframe (yes cleanup)
        where we get bank's balance sheet info
    model: model for default probability
        scikit learn LogisticRegression
    variables: a list of strings
        independent variables for the model
    
    Returns
    ----------
    WN: 3D numpy array [n_simulations, n_periods, n_banks]
        array of predicted probability of default
    """
    
    simNum, periodNum, bankNum, _= N.shape
    WN = np.empty((simNum, periodNum, bankNum))
    WN.fill(-1)
    
    for s in range(simNum):
        for p in range(2,periodNum):
            for b in range(bankNum):
                X = dff[np.array(dff["sim#"]==s) &
                        np.array(dff["period"]==p) & 
                        np.array(dff["bankID"]==b)
                       ][variables].values
                if X.any():
                    predicted_default_probability = model.predict_proba(X)[0][1]
                    WN[s, p-1, b] = predicted_default_probability
                    
    return WN

In [13]:
edge_weights_all = create_edge_weight(mx_1n, df_1, 0.6)
node_weights_all = create_node_weight(mx_1n, df_1c, final, independent)

Empirically, I discover that it does not matter 0.6 is. I experimented with serveral numbers between 0 and 1.

In [14]:
# visualize the debt adjacency matrix in a 2d graph
# plt.figure(figsize=(10,10))
# plt.imshow(edge_weights_all[0,1], interpolation='nearest')
# plt.show()

In [15]:
def random_walk_single_1a(N, solvent, iterations=10):
    """
    NEW NEW modified random walk algorithm
    at the begining of each round, flip coin to cause one solvent bank to default
    
    This one is aggressive
    [[18054    25] ---0625/1IR
     [  158     3]]
    [[17926    30] ---0625/2IR
     [  160     3]]
    
    Parameters
    ----------
    N: 2D numpy array [n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: list 
        solvent[solvent bank] = predicted default probability
        solvent[insolvent bank] = -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: dict {BankID : # of default}
        index for the bank's default probability
    """
    num_bank, _ = N.shape
    G = nx.DiGraph(N)
    nextDefault = deque()
    defaulted = frozenset([i for i in range(num_bank) if solvent[i] == -1])
    tempDefault = set(defaulted)
    probDefault = {i:solvent[i] for i in range(num_bank) if solvent[i] >= 0}
    result = dict(zip(list(probDefault.keys()), [0 for _ in probDefault.keys()]))
    def coin(p=0.5):
        return random() < p
    
    for _ in range(iterations):
        for b in probDefault.keys():

            if coin(probDefault[b]): 
                nextDefault.append(b)
            while len(nextDefault) > 0: # queue not empty
                n = nextDefault.popleft() # next on the queue
                if n not in tempDefault: # not aleady defaulted
                    tempDefault.add(n)
                    result[n] += 1
                    for s in G.successors(n): # creditors of n
                        if s not in tempDefault and coin(G.edges[(n,s)]['weight']):
                            nextDefault.append(s)
            tempDefault = set(defaulted)
            
    return result

In [16]:
def random_walk_single_1b(N, solvent, iterations=10):
    """
    NEW NEW modified random walk algorithm
    at the begining of each round, flip coin to cause one solvent bank to default
    during each round, we can add 1 multiple times
    
    This one is also aggressive
    [[18052    27] ---0625/1IR
     [  157     4]]
    [[17927    29] ---0625/2IR
     [  160     3]]
    
    Parameters
    ----------
    N: 2D numpy array [n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: list 
        solvent[solvent bank] = predicted default probability
        solvent[insolvent bank] = -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: dict {BankID : # of default}
        index for the bank's default probability
    """
    num_bank, _ = N.shape
    G = nx.DiGraph(N)
    nextDefault = deque()
    probDefault = {i:solvent[i] for i in range(num_bank) if solvent[i] >= 0}
    result = dict(zip(list(probDefault.keys()), [0 for _ in probDefault.keys()]))
    def coin(p=0.5):
        return random() < p
    
    for _ in range(iterations):
        for b in probDefault.keys():
            if coin(probDefault[b]): 
                nextDefault.append(b)
            while len(nextDefault) > 0: # queue not empty
                n = nextDefault.popleft() # next on the queue
                result[n] += 1 # default count ++
                for s in G.successors(n): # creditors of n
                    if coin(G.edges[(n,s)]['weight']):
                        nextDefault.append(s)
                        
    return result

In [17]:
def random_walk_single_2a(N, solvent, iterations=10):
    """
    NEW NEW NEW modified random walk algorithm
    at the begining of each round, flip multiple coins for each solvent bank to make them default
    
    This one is useless (very progressive):
    [[18079     0] ---0625/1IR
     [  161     0]]
    [[17956     0] ---0625/2IR
     [  163     0]]
    
    Parameters
    ----------
    N: 2D numpy array [n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: list 
        solvent[solvent bank] = predicted default probability
        solvent[insolvent bank] = -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: dict {BankID : # of default}
        index for the bank's default probability
    """
    num_bank, _ = N.shape
    G = nx.DiGraph(N)
    nextDefault = deque()
    defaulted = frozenset([i for i in range(num_bank) if solvent[i] == -1])
    tempDefault = set(defaulted)
    probDefault = {i:solvent[i] for i in range(num_bank) if solvent[i] >= 0}
    result = dict(zip(list(probDefault.keys()), [0 for _ in probDefault.keys()]))
    def coin(p=0.5):
        return random() < p
    
    for _ in range(iterations):
        for b in probDefault.keys():
            if coin(probDefault[b]): 
                nextDefault.append(b)

        while len(nextDefault) > 0: # queue not empty
            n = nextDefault.popleft() # next on the queue
            if n not in tempDefault: # not aleady defaulted
                tempDefault.add(n)
                result[n] += 1
                for s in G.successors(n): # creditors of n
                    if s not in tempDefault and coin(G.edges[(n,s)]['weight']):
                        nextDefault.append(s)
        tempDefault = set(defaulted)
    
    return result

In [18]:
def random_walk_single_2b(N, solvent, iterations=10):
    """
    NEW NEW NEW modified random walk algorithm
    at the begining of each round, flip multiple coins for each solvent bank to make them default
    during each round, we can add 1 multiple times
    
    This one is progressive:
    [[18048    31] ---0625/1IR
     [  157     4]]
    [[17926    30] ---0625/2IR
     [  157     6]]
    
    Parameters
    ----------
    N: 2D numpy array [n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: list 
        solvent[solvent bank] = predicted default probability
        solvent[insolvent bank] = -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: dict {BankID : # of default}
        index for the bank's default probability
    """
    num_bank, _ = N.shape
    G = nx.DiGraph(N)
    nextDefault = deque()
    probDefault = {i:solvent[i] for i in range(num_bank) if solvent[i] >= 0}
    result = dict(zip(list(probDefault.keys()), [0 for _ in probDefault.keys()]))
    def coin(p=0.5):
        return random() < p
    
    for _ in range(iterations):
        for b in probDefault.keys():
            if coin(probDefault[b]): 
                nextDefault.append(b)
        while len(nextDefault) > 0: # queue not empty
            n = nextDefault.popleft() # next on the queue
            result[n] += 1
            for s in G.successors(n): # creditors of n
                if coin(G.edges[(n,s)]['weight']):
                    nextDefault.append(s)
                    
    return result

In [19]:
def customized_random_walk_exp(N, solvent, random_walk, iterations=10):
    """
    NEW NEW NEW modified random walk algorithm
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    solvent: 3D numpy array [n_simulations, n_periods, n_borrowers, n_lenders] 
        predicted default probability OR -1
    iterations: int
        number of iterations
    
    Returns
    ----------
    result: 1D numpy array
        index for the bank's default probability   
    """
    n_s, n_p, n_b, _ = N.shape
    s_s, s_p, s_b = solvent.shape
    big_result = []
    
    if n_s != s_s or n_p != s_p or n_b != s_b:
        raise ValueError('Two arrays have incompatible sizes.')
        
    for i, j in np.ndindex((n_s,n_p)):
        big_result.extend(random_walk(N[i,j],
                                      solvent[i,j],
                                      iterations=iterations
                                     ).values())
        
    return np.array(big_result)

In [20]:
overall_prediction = customized_random_walk_exp(edge_weights_all, node_weights_all, random_walk_single_1b, 500)

In [142]:
len(overall_prediction)

18240

In [143]:
# ###########################
# pearson correlation coeffcient is not very big 
# it gets slightly bigger with lots of iterations
# ###########################
pearsonr(overall_prediction, df_1c["default-next"])

(0.29103589661409535, 0.0)

In [144]:
df_1c.groupby("default-next").count()

,sim#,period,bankID,theta (risk aversion),wealth,deposits,cash,assets,leverage,credit available,...,cash-lag,assets-lag,leverage-lag,credit-available-lag,credit-issued-lag,dummy-0-leverage-lag,over-leverage-frequency,default-next-wealth,default-next-deposit,default-next-interest
default-next,,,,,,,,,,,,,,,,,,,,,
0,18079,18079,18079,18079,18079,18079,18079,18079,18079,18079,...,18079,18079,18079,18079,18079,18079,18079,18079,18079,18079
1,161,161,161,161,161,161,161,161,161,161,...,161,161,161,161,161,161,161,161,161,161


In [23]:
yyy = df_1c["default-next"]
# np.array(range(len(yyy)))[yyy==0]
np.random.choice(np.array(range(len(yyy)))[yyy==1],size=100,replace=False)

array([ 8275,  4737, 17100, 16427,  1783, 14057,  1068, 16419,  5574,
       11613,  2077, 16455,  2072,   603, 16420,  6913,  5545,   377,
        9470, 14082,  1070,  8271, 12436, 11976,  7564,  9394,  1811,
        9047, 16421,  9683,  2074, 11978,  6623,    93,  9045,  6911,
        9393, 17096,  6938,  8821,  3278,  4315,  9424,  3797,  2485,
        7300,  5544,  9303, 11167,  2070, 11947,  9049,  9427,  1067,
       11168, 14567, 15800, 17095,  6172,  6910,  8706, 12435,   374,
       16141,   376,  9423,   378,    90,  9429,  1069, 16283,  9421,
        9425, 11170,  6173,  8273, 11858,  8276, 11946,  2486,  6565,
        1815,  6167,  8793,   373,  1814, 14568, 16734,  9681, 16454,
        2071, 16453,  4739, 17099,  2073,  6174,  4374,  6169,  2455,
        4318])

In [147]:
def customized_k_fold(X, y, fold=12):
    """
    customized K-fold cross validation 
    print certain summary statistics
    """
    # k-fold
    kf = KFold(n_splits=fold, shuffle=True)

    # initialization
    accuracy = 0
    conf = np.array([[0, 0], [0, 0]])

    def g(A, arr):
        if A.shape[1] == 1: return A[arr]
        else: return A.loc[arr]
    
    for train_index, test_index in kf.split(X):
        model = LogisticRegression()
        model.fit(g(X,train_index), y.iloc[train_index])
        print(model.coef_, model.intercept_)

        accuracy += model.score(g(X,test_index), y.iloc[test_index])
        conf += confusion_matrix(y.iloc[test_index], model.predict(g(X,test_index)))

    tn, fp, fn, tp = conf.ravel()
    print("{}\n accuracy:{:24}\n precision:{:24}\n recall:{:24}\n".format(
            conf, 
            accuracy/fold, 
            tp/(tp+fp),
            tp/(tp+fn)))

In [150]:
def customized_k_fold_balanced(X, y, fold=12):
    """
    customized K-fold cross validation 
    print certain summary statistics
    """
    all_obs = np.array(range(len(y)))
    default_obs = all_obs[y==1]
    nondefault_obs = np.random.choice(all_obs[y==0],
                                     size=len(default_obs),
                                     replace=False)
    all_index = np.append(default_obs, nondefault_obs)
    customized_k_fold(X[all_index], y[all_index], fold=12)

In [151]:
# 1111111111111111111111111
# K-fold cross validation 
# use approx. default probability to predict default
# ###########################
customized_k_fold_balanced(overall_prediction.reshape(-1, 1), df_1c["default-next"])

[[0.01240397]] [-3.60380092]
[[0.01151696]] [-3.44328164]
[[0.01195532]] [-3.56537264]
[[0.01144232]] [-3.42357864]
[[0.01143624]] [-3.43655865]
[[0.01160685]] [-3.40815917]
[[0.01153255]] [-3.41380608]
[[0.01169909]] [-3.53273655]
[[0.01181041]] [-3.45602798]
[[0.01176579]] [-3.52899681]
[[0.01218029]] [-3.59629783]
[[0.01213345]] [-3.54771385]
[[150  11]
 [  7 154]]
 accuracy:      0.9440883190883191
 precision:      0.9333333333333333
 recall:      0.9565217391304348



In [127]:
# prediction based on logistic regression with balanced sheet information
bs_prediction_all = node_weights_all.reshape(1,-1)[0]
bal_sh_prediction = bs_prediction_all [bs_prediction_all != -1]

In [128]:
# zip two predictions into a dataframe
two_predictions = pd.DataFrame({'balanced sheet':bal_sh_prediction, 'network':overall_prediction})

In [129]:
# 222222222222222222222222222
# K-fold cross validation
# use approx. default probability + balanced sheet default prob to predict default
# This does not seem to be an effective approach
# ###########################
customized_k_fold_balanced(two_predictions, df_1c["default-next"])

KeyError: '[   90    93   345   346   372   373   374   376   377   378   603   734\n   820   903  1067  1068  1069  1070  1071  1143  1722  1783  1811  1813\n  1814  1815  1817  1818  2070  2071  2072  2073  2074  2075  2076  2077\n  2455  2485  2486  2540  3218  3249  3250  3278  3600  3797  4315  4318\n  4374  4737  4739  5544  5545  5574  6167  6168  6169  6170  6171  6172\n  6173  6174  6565  6623  6909  6910  6911  6912  6913  6938  7300  7302\n  7329  7330  7331  7563  7564  7565  7567  7592  8271  8272  8273  8275\n  8276  8706  8793  8821  9044  9045  9047  9049  9152  9303  9393  9394\n  9421  9423  9424  9425  9427  9428  9429  9430  9431  9457  9470  9681\n  9683  9708 10079 10080 11167 11168 11170 11171 11612 11613 11614 11858\n 11946 11947 11948 11975 11976 11977 11978 12405 12433 12434 12435 12436\n 13822 14027 14057 14082 14110 14567 14568 14597 14598 15800 15830 15942\n 16141 16142 16283 16419 16420 16421 16424 16427 16453 16454 16455 16734\n 17095 17096 17099 17100 17

Combining (logsitic regression) balanced sheet default prediction (logisitis regression) with network information (4 modified page rank I created) does not seem to be effective. The result predicts no default, only not default. However, the accuracy is also very high (> 99%).

----------------------------------------------------------------------
Stuff Below this are old stuff that I might or might not need.
----------------------------------------------------------------------
----------------------------------------------------------------------

In [125]:
def dist_avg_max(N):
    """
    calculate average & max distances between all pair of nodes
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    
    Returns
    ----------
    avg_d: 2D numpy array [n_simulations, n_periods]
        average distances between all pair of nodes
    max_d: 2D numpy array [n_simulations, n_periods]
        max distances between all pair of nodes
    """
    numSim, numPeriod, _, _ = N.shape
    avg_d = np.empty((numSim, numPeriod-2))
    max_d = np.empty((numSim, numPeriod-2))
    
    for s in range(numSim):
        for p in range(1,numPeriod-1):
            disG = nx.DiGraph(N[s,p])
            dists = shortest_path_length(disG, weight=None)
            curlist=[]
            for source in dists:
                curlist.extend(source[1].values())
            avg_d[s,p-1] = sum(curlist) / float(len(curlist))
            max_d[s,p-1] = max(curlist)
            
    return avg_d, max_d

In [120]:
# ###########################
# Visualize max/avg distances between banks
# ###########################
# avgg, maxx = dist_avg_max(mx_1n)
#
# pavg = pd.DataFrame(avgg)
# pmax = pd.DataFrame(maxx)
# # pavg.mean().plot()
# abc = pmax.stack().value_counts().sort_index().plot(
#     kind="bar",
#     title="max distances, 1 interest rates, 50 simulations, 15 periods",
#     figsize=(8,6),
#     fontsize=12
# )
# abc.set_xlabel("max distance between any pair of reachable nodes")
# abc.set_ylabel("frequncy")
# abc.title.set_fontsize(15)
# abc.xaxis.label.set_fontsize(15)
# abc.yaxis.label.set_fontsize(15)

In [6]:
def weigh_networks(N, model, variables):
    """
    Add weight to network
    Each debt is multiplied by lenders' predicted default probability
    
    Parameters
    ----------
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices 
    model: scikit learn LogisticRegression
        model for default probability
    variables: a list of strings
        independent variables for the model
    
    Returns
    ----------
    WN: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        new weighted debt adjacency matrices
    """
    
    WN = np.copy(N)
    simNum, periodNum, bankNum, _= N.shape
    
    for s in range(simNum):
        for p in range(1,periodNum-1):
            for b in range(bankNum):
                X = df_1c[np.array(df_1c["sim#"]==s) &
                          np.array(df_1c["period"]==p) & 
                          np.array(df_1c["bankID"]==b)
                         ][variables].values
                if X.any():
                    predicted_default_probability = model.predict_proba(X)[0][1]
                    WN[s, p-1, b] *= predicted_default_probability
                    
    return WN

In [46]:
def my_pagerank_numpy(G, alpha=0.85, personalization=None, weight='weight', dangling=None):
    """
    This is basically pagerank_numpy without normalization.
    """
    from networkx.algorithms.link_analysis.pagerank_alg import google_matrix
    
    if len(G) == 0:
        return {}
    M = google_matrix(G, alpha, personalization=personalization,
                      weight=weight, dangling=dangling)
    # use numpy LAPACK solver
    eigenvalues, eigenvectors = np.linalg.eig(M.T)
    ind = np.argmax(eigenvalues)
    # eigenvector of largest eigenvalue is at ind
    largest = np.array(eigenvectors[:, ind]).flatten().real
    return dict(zip(G, map(float, abs(largest))))

In [8]:
def apply_to_networks(f, N):
    """
    Calculate Page Rank scores for all the networks 
    
    Parameters
    ----------
    f: function (2D numpy array -> matrix)
        the function to apply to each network (e.g. Page Rank)
    N: 4D numpy array [n_simulations, n_periods, n_borrowers, n_lenders]
        debt adjacency matrices (netowrks)
    
    Returns
    ----------
    PG: 3D numpy array [n_simulations, n_periods, n_banks]
        Page Rank scores
    """
    
    simNum, periodNum, bankNum, _= N.shape
    PG = np.empty((simNum, periodNum, bankNum))
    
    for s in range(simNum):
        for p in range(1,periodNum-1):
            PG[s, p] = np.array(list(f(nx.DiGraph(N[s, p])).values()))
            
    return PG

In [9]:
# ###########################
# Let's add the weight
# ###########################
weighted = weigh_networks(mx_1n, final, independent)

In [50]:
# ###########################
# Let's calculate pagerank
# ###########################
pg_iter = apply_to_networks(pagerank, weighted)
pg_norm = apply_to_networks(pagerank_numpy, weighted)
pg_not_norm = apply_to_networks(my_pagerank_numpy, weighted)